In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
print(df.shape)
print(df.columns)
df.head()


In [ ]:
positive=0
neg=0
import seaborn as sns
import matplotlib.pyplot as plt
for i in range(df.shape[0]):
    if df.loc[i,"Class"] == 1:
        positive=positive+1
    else:
        neg=neg+1
        
print("Positive class size : " , positive)
print("Negative class size : ",neg)


sns.catplot(x='Class',kind='count', data=df);




In [ ]:
df.hist(bins=50,grid=False,figsize=(200,200),xlabelsize=100,ylabelsize=100)



In [ ]:
from sklearn import preprocessing
scaler = preprocessing.RobustScaler()
df["Scaled_Amount"]=scaler.fit_transform(df["Amount"].values.reshape(-1,1))
df["Scaled_Time"]=scaler.fit_transform(df["Time"].values.reshape(-1,1))
df=df.drop(["Amount","Time"],axis=1)


In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
y=df["Class"]
X=df.drop(["Class"],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=None)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# training and testing without undersampling
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

log_reg_default=LogisticRegression()
log_reg_default.fit(X_train, y_train)
y_pred=log_reg_default.predict(X_test)

print("Logistic Regression before any tuning of parameter : ")
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))





In [ ]:
#undersampling of data
new_df=pd.concat([X_train,y_train],axis=1)
new_df=new_df.sample(frac=1)
undersample_data_positive=new_df.loc[new_df["Class"]==1]
undersample_data_negative=new_df.loc[new_df["Class"]==0][:undersample_data_positive.shape[0]]
new_df=pd.concat([undersample_data_positive,undersample_data_negative])
new_df=new_df.sample(frac=1)
new_df

In [ ]:
corr=new_df.corr()
f, ax = plt.subplots(figsize=(10,10))
ax = sns.heatmap(corr, center=0)


In [ ]:
f,axes = plt.subplots(ncols=5,figsize=(50,20))
sns.boxplot(x="Class",y="V17",data=new_df,ax=axes[0])
axes[0].set_title('V17 vs Class negative correlation')
sns.boxplot(x="Class",y="V14",data=new_df,ax=axes[1])
axes[1].set_title('V14 vs Class negative correlation')
sns.boxplot(x="Class",y="V12",data=new_df,ax=axes[2])
axes[2].set_title('V12 vs Class negative correlation')
sns.boxplot(x="Class",y="V10",data=new_df,ax=axes[3])
axes[3].set_title('V10 vs Class negative correlation')
sns.boxplot(x="Class",y="V16",data=new_df,ax=axes[4])
axes[4].set_title('V16 vs Class negative correlation')

In [ ]:
f,axes = plt.subplots(ncols=4,figsize=(50,20))
sns.boxplot(x="Class",y="V11",data=new_df,ax=axes[0])
axes[0].set_title('V11 vs Class negative correlation')
sns.boxplot(x="Class",y="V4",data=new_df,ax=axes[1])
axes[0].set_title('V4 vs Class negative correlation')
sns.boxplot(x="Class",y="V2",data=new_df,ax=axes[2])
axes[0].set_title('V2 vs Class negative correlation')
sns.boxplot(x="Class",y="V19",data=new_df,ax=axes[3])
axes[0].set_title('V19 vs Class negative correlation')

In [ ]:
v17_values = new_df["V17"].values
q25,q75 = np.percentile(v17_values,25),np.percentile(v17_values,75)
iqr=q75-q25
v17_cutoff = iqr*1.5
v17_upper=q75+v17_cutoff
v17_lower=q25-v17_cutoff
new_df=new_df.drop(new_df[(new_df["V17"] > v17_upper) | (new_df["V17"] < v17_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V17_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v17_upper)
print('lower : ',v17_lower)

v14_values = new_df["V14"].values
q25,q75 = np.percentile(v14_values,25),np.percentile(v14_values,75)
iqr=q75-q25
v14_cutoff = iqr*1.5
v14_upper=q75+v14_cutoff
v14_lower=q25-v14_cutoff
new_df=new_df.drop(new_df[(new_df["V14"] > v14_upper) | (new_df["V14"] < v14_lower)].index)

print('V14_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v14_upper)
print('lower : ',v14_lower)

v12_values = new_df["V12"].values
q25,q75 = np.percentile(v12_values,25),np.percentile(v12_values,75)
iqr=q75-q25
v12_cutoff = iqr*1.5
v12_upper=q75+v12_cutoff
v12_lower=q25-v12_cutoff
new_df=new_df.drop(new_df[(new_df["V12"] > v12_upper) | (new_df["V12"] < v12_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V12_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v12_upper)
print('lower : ',v12_lower)

v10_values = new_df["V10"].values
q25,q75 = np.percentile(v10_values,25),np.percentile(v10_values,75)
iqr=q75-q25
v10_cutoff = iqr*1.5
v10_upper=q75+v10_cutoff
v10_lower=q25-v10_cutoff
new_df=new_df.drop(new_df[(new_df["V10"] > v10_upper) | (new_df["V10"] < v10_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V10_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v10_upper)
print('lower : ',v10_lower)

v16_values = new_df["V16"].values
q25,q75 = np.percentile(v16_values,25),np.percentile(v16_values,75)
iqr=q75-q25
v16_cutoff = iqr*1.5
v16_upper=q75+v16_cutoff
v16_lower=q25-v16_cutoff
new_df=new_df.drop(new_df[(new_df["V16"] > v16_upper) | (new_df["V16"] < v16_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V16_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v16_upper)
print('lower : ',v16_lower)

In [ ]:
v11_values = new_df["V11"].values
q25,q75 = np.percentile(v11_values,25),np.percentile(v11_values,75)
iqr=q75-q25
v11_cutoff = iqr*1.5
v11_upper=q75+v11_cutoff
v11_lower=q25-v11_cutoff
new_df=new_df.drop(new_df[(new_df["V11"] > v11_upper) | (new_df["V11"] < v11_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V11_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v11_upper)
print('lower : ',v11_lower)

v4_values = new_df["V4"].values
q25,q75 = np.percentile(v4_values,25),np.percentile(v4_values,75)
iqr=q75-q25
v4_cutoff = iqr*1.5
v4_upper=q75+v4_cutoff
v4_lower=q25-v4_cutoff
new_df=new_df.drop(new_df[(new_df["V4"] > v4_upper) | (new_df["V4"] < v4_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V4_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v4_upper)
print('lower : ',v4_lower)

v2_values = new_df["V2"].values
q25,q75 = np.percentile(v2_values,25),np.percentile(v2_values,75)
iqr=q75-q25
v2_cutoff = iqr*1.5
v2_upper=q75+v2_cutoff
v2_lower=q25-v2_cutoff
new_df=new_df.drop(new_df[(new_df["V2"] > v2_upper) | (new_df["V2"] < v2_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V2_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v2_upper)
print('lower : ',v2_lower)


v19_values = new_df["V19"].values
q25,q75 = np.percentile(v19_values,25),np.percentile(v19_values,75)
iqr=q75-q25
v19_cutoff = iqr*1.5
v19_upper=q75+v19_cutoff
v19_lower=q25-v19_cutoff
new_df=new_df.drop(new_df[(new_df["V19"] > v19_upper) | (new_df["V19"] < v19_lower)].index)
print('Number of rows after outliers are removed : ',new_df.shape[0])
print('V19_values_stats')
print('q25 : ',q25)
print('q75 : ',q75)
print('iqr : ',iqr)
print('upper : ',v19_upper)
print('lower : ',v19_lower)



In [ ]:
y_train=new_df["Class"]
X_train=new_df.drop(["Class"],axis=1)
# X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=None)
# print(X_train.shape)
# print(X_val.shape)
# print(y_train.shape)
# print(y_val.shape)

In [ ]:
log_reg_default=LogisticRegression()
log_reg_default.fit(X_train, y_train)
y_pred=log_reg_default.predict(X_test)

print("Logistic Regression before any tuning of parameter : ")
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

KNeigh = KNeighborsClassifier()
KNeigh.fit(X_train,y_train)
y_pred=KNeigh.predict(X_test)
print("KNearest Neighbours classifier before any tuning of parameter : ")
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))


svc = SVC()
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print("Support Vector classifier before any tuning of parameter : ")
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))


In [ ]:

#Logistic regression
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params)
grid_log_reg.fit(X_train, y_train)

log_reg = grid_log_reg.best_estimator_
print(log_reg)

log_reg_score = cross_val_score(log_reg, X_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')

y_pred=log_reg.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

# Knearest neigbour

knears_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}

grid_knears = GridSearchCV(KNeighborsClassifier(), knears_params)
grid_knears.fit(X_train, y_train)
# KNears best estimator
knears_neighbors = grid_knears.best_estimator_
print(knears_neighbors)
knears_score = cross_val_score(knears_neighbors, X_train, y_train, cv=5)
print('Knears Neighbors Cross Validation Score', round(knears_score.mean() * 100, 2).astype(str) + '%')

y_pred=knears_neighbors.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))


# Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
grid_svc = GridSearchCV(SVC(), svc_params)
grid_svc.fit(X_train, y_train)

# SVC best estimator
svc = grid_svc.best_estimator_
print(svc)
y_pred=svc.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))

# DecisionTree Classifier
tree_params = {"criterion": ["gini", "entropy"], "max_depth": list(range(2,6,1,)), 
              "min_samples_leaf": list(range(5,20,2))}
grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_params)
grid_tree.fit(X_train, y_train)

# tree best estimator
tree_clf = grid_tree.best_estimator_
print(tree_clf)
y_pred=tree_clf.predict(X_test)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred)))
print('Precision Score : ' + str(precision_score(y_test,y_pred)))
print('Recall Score : ' + str(recall_score(y_test,y_pred)))
print('F1 Score : ' + str(f1_score(y_test,y_pred)))

from sklearn.metrics import confusion_matrix
print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))



In [ ]:
#learning curves
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
cv=ShuffleSplit(n_splits=5,random_state=None,test_size=0.25,train_size=None)
print(cv)
f, axes = plt.subplots(ncols=2,figsize=(30,20))
axes[0].set_title("Logistics regression learning curves")
train_size,train_score,cross_val_score = learning_curve(log_reg,X_train,y_train,cv=cv,n_jobs=4,train_sizes=np.linspace(.1, 1.0, 5))
print(train_size)
print(train_score)
print(cross_val_score)
train_scores_mean = np.mean(train_score, axis=1)
train_scores_std = np.std(train_score, axis=1)
cross_val_scores_mean = np.mean(cross_val_score, axis=1)
cross_val_scores_std = np.std(cross_val_score, axis=1)
axes[0].plot(train_size,train_scores_mean)
axes[0].plot(train_size,cross_val_scores_mean)


axes[1].set_title("Support Vector machine learning curves")
train_size,train_score,cross_val_score = learning_curve(svc,X_train,y_train,cv=cv,n_jobs=4,train_sizes=np.linspace(.1, 1.0, 5))
print(train_size)
print(train_score)
print(cross_val_score)
train_scores_mean = np.mean(train_score, axis=1)
train_scores_std = np.std(train_score, axis=1)
cross_val_scores_mean = np.mean(cross_val_score, axis=1)
cross_val_scores_std = np.std(cross_val_score, axis=1)
axes[1].plot(train_size,train_scores_mean)
axes[1].plot(train_size,cross_val_scores_mean)

f, axes = plt.subplots(ncols=2,figsize=(30,20))

axes[0].set_title("KNearest Neighbors classifier learning curves")
train_size,train_score,cross_val_score = learning_curve(knears_neighbors,X_train,y_train,cv=cv,n_jobs=4,train_sizes=np.linspace(.1, 1.0, 5))
print(train_size)
print(train_score)
print(cross_val_score)
train_scores_mean = np.mean(train_score, axis=1)
train_scores_std = np.std(train_score, axis=1)
cross_val_scores_mean = np.mean(cross_val_score, axis=1)
cross_val_scores_std = np.std(cross_val_score, axis=1)
axes[0].plot(train_size,train_scores_mean)
axes[0].plot(train_size,cross_val_scores_mean)

axes[1].set_title("decision tree learning curves")
train_size,train_score,cross_val_score = learning_curve(tree_clf,X_train,y_train,cv=cv,n_jobs=4,train_sizes=np.linspace(.1, 1.0, 5))
print(train_size)
print(train_score)
print(cross_val_score)
train_scores_mean = np.mean(train_score, axis=1)
train_scores_std = np.std(train_score, axis=1)
cross_val_scores_mean = np.mean(cross_val_score, axis=1)
cross_val_scores_std = np.std(cross_val_score, axis=1)
axes[1].plot(train_size,train_scores_mean)
axes[1].plot(train_size,cross_val_scores_mean)
